# खर्च दावा विश्लेषण

यह नोटबुक दिखाती है कि प्लगइन्स का उपयोग करके एजेंट्स कैसे बनाए जाते हैं, जो स्थानीय रसीद छवियों से यात्रा खर्चों को प्रोसेस करते हैं, एक खर्च दावा ईमेल तैयार करते हैं, और पाई चार्ट का उपयोग करके खर्च डेटा को विज़ुअलाइज़ करते हैं। एजेंट्स कार्य के संदर्भ के आधार पर डायनामिक रूप से फंक्शन्स का चयन करते हैं।

चरण:
1. OCR एजेंट स्थानीय रसीद छवि को प्रोसेस करता है और यात्रा खर्च डेटा को निकालता है।
2. ईमेल एजेंट एक खर्च दावा ईमेल तैयार करता है।

### यात्रा खर्च परिदृश्य का उदाहरण:
कल्पना करें कि आप एक कर्मचारी हैं जो किसी अन्य शहर में एक व्यावसायिक बैठक के लिए यात्रा कर रहे हैं। आपकी कंपनी की नीति है कि सभी उचित यात्रा-संबंधित खर्चों की प्रतिपूर्ति की जाएगी। यहां संभावित यात्रा खर्चों का विवरण दिया गया है:
- परिवहन:
आपके घर के शहर से गंतव्य शहर तक राउंड ट्रिप के लिए हवाई किराया।
हवाई अड्डे तक और वहां से टैक्सी या राइड-हेलिंग सेवाएं।
गंतव्य शहर में स्थानीय परिवहन (जैसे सार्वजनिक परिवहन, किराए की कारें, या टैक्सी)।

- आवास:
बैठक स्थल के पास एक मिड-रेंज बिजनेस होटल में तीन रातों का ठहराव।

- भोजन:
कंपनी की दैनिक भत्ता नीति के आधार पर नाश्ता, दोपहर का भोजन, और रात के खाने के लिए दैनिक भोजन भत्ता।

- विविध खर्च:
हवाई अड्डे पर पार्किंग शुल्क।
होटल में इंटरनेट एक्सेस शुल्क।
टिप्स या छोटे सेवा शुल्क।

- दस्तावेज़ीकरण:
आप प्रतिपूर्ति के लिए सभी रसीदें (फ्लाइट्स, टैक्सी, होटल, भोजन आदि) और एक पूरा किया हुआ खर्च रिपोर्ट जमा करते हैं।


## आवश्यक लाइब्रेरी आयात करें

नोटबुक के लिए आवश्यक लाइब्रेरी और मॉड्यूल आयात करें।


In [1]:
import os
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from openai import AsyncOpenAI
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat


from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.strategies import SequentialSelectionStrategy, DefaultTerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import ImageContent, TextContent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings

from semantic_kernel.functions import kernel_function, KernelArguments
from pydantic import BaseModel, Field
from typing import List
from azure.ai.inference.models import SystemMessage, UserMessage, TextContentItem, ImageContentItem, ImageUrl, ImageDetailLevel

load_dotenv()

True

In [2]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
   
    client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"], base_url="https://models.inference.ai.azure.com/")
    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o-mini",
            async_client=client,
            service_id="open_ai"
        )
    )

    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o",
            async_client=client,
            service_id="gpt-4o"
        )
    )

    return kernel

## खर्च मॉडल परिभाषित करें

 व्यक्तिगत खर्चों के लिए एक Pydantic मॉडल बनाएं और एक ExpenseFormatter क्लास तैयार करें जो उपयोगकर्ता की क्वेरी को संरचित खर्च डेटा में बदल सके।

 प्रत्येक खर्च को निम्नलिखित प्रारूप में प्रस्तुत किया जाएगा:
 `{'date': '07-Mar-2025', 'description': 'गंतव्य के लिए उड़ान', 'amount': 675.99, 'category': 'Transportation'}`


In [3]:
class Expense(BaseModel):
    date: str = Field(..., description="Date of expense in dd-MMM-yyyy format")
    description: str = Field(..., description="Expense description")
    amount: float = Field(..., description="Expense amount")
    category: str = Field(..., description="Expense category (e.g., Transportation, Meals, Accommodation, Miscellaneous)")

class ExpenseFormatter(BaseModel):
    raw_query: str = Field(..., description="Raw query input containing expense details")
    
    def parse_expenses(self) -> List[Expense]:
        """
        Parses the raw query into a list of Expense objects.
        Expected format: "date|description|amount|category" separated by semicolons.
        """
        expense_list = []
        for expense_str in self.raw_query.split(";"):
            if expense_str.strip():
                parts = expense_str.strip().split("|")
                if len(parts) == 4:
                    date, description, amount, category = parts
                    try:
                        expense = Expense(
                            date=date.strip(),
                            description=description.strip(),
                            amount=float(amount.strip()),
                            category=category.strip()
                        )
                        expense_list.append(expense)
                    except ValueError as e:
                        print(f"[LOG] Parse Error: Invalid data in '{expense_str}': {e}")
        return expense_list

## एजेंट्स को परिभाषित करना - ईमेल तैयार करना

एक एजेंट क्लास बनाएं जो खर्चा दावा सबमिट करने के लिए एक ईमेल तैयार करे।  
- यह एजेंट `kernel_function` डेकोरेटर का उपयोग करता है ताकि एक ऐसा फंक्शन परिभाषित किया जा सके जो खर्चा दावा सबमिट करने के लिए ईमेल तैयार करे।  
- यह खर्चों की कुल राशि की गणना करता है और विवरण को ईमेल बॉडी में स्वरूपित करता है।  


In [4]:
class ExpenseEmailAgent:

    @kernel_function(description="Generate an email to submit an expense claim to the Finance Team")
    async def generate_expense_email(expenses):
        total_amount = sum(expense['amount'] for expense in expenses)
        email_body = "Dear Finance Team,\n\n"
        email_body += "Please find below the details of my expense claim:\n\n"
        for expense in expenses:
            email_body += f"- {expense['description']}: ${expense['amount']}\n"
        email_body += f"\nTotal Amount: ${total_amount}\n\n"
        email_body += "Receipts for all expenses are attached for your reference.\n\n"
        email_body += "Thank you,\n[Your Name]"
        return email_body

# रसीद छवियों से यात्रा खर्च निकालने के लिए एजेंट

रसीद छवियों से यात्रा खर्च निकालने के लिए एक एजेंट क्लास बनाएं।  
- यह एजेंट यात्रा खर्च निकालने के लिए एक फ़ंक्शन को परिभाषित करने के लिए `kernel_function` डेकोरेटर का उपयोग करता है।  
- रसीद छवि को OCR (ऑप्टिकल कैरेक्टर रिकग्निशन) का उपयोग करके टेक्स्ट में बदलें और प्रासंगिक जानकारी जैसे तारीख, विवरण, राशि, और श्रेणी को निकालें।  


In [5]:
class OCRAgentPlugin:
    def __init__(self):
        self.client = ChatCompletionsClient(
            endpoint="https://models.inference.ai.azure.com/",
            credential=AzureKeyCredential(os.environ.get("GITHUB_TOKEN")),
        )
        self.model_name = "gpt-4o"

    @kernel_function(description="Extract structured travel expense data from receipt.jpg using gpt-4o-model")
    def extract_text(self, image_path: str = "receipt.jpg") -> str:
        try:
            image_url_str = str(ImageUrl.load(image_file=image_path, image_format="jpg", detail=ImageDetailLevel.HIGH))

            prompt = (
                "You are an expert OCR assistant specialized in extracting structured data from receipt images. "
                "Analyze the provided receipt image and extract travel-related expense details in the format: "
                "'date|description|amount|category' separated by semicolons. "
                "Follow these rules: "
                "- Date: Convert dates (e.g., '4/4/22') to 'dd-MMM-yyyy' (e.g., '04-Apr-2022'). "
                "- Description: Extract item names (e.g., 'Carlson's Drylawn', 'Peigs transaction Probiotics'). "
                "- Amount: Use numeric values (e.g., '4.50' from '$4.50' or '4.50 dollars'). "
                "- Category: Infer from context (e.g., 'Meals' for food, 'Transportation' for travel, 'Accommodation' for lodging, 'Miscellaneous' otherwise). "
                "Ignore totals, subtotals, or service charges unless they are itemized expenses. "
                "If no expenses are found, return 'No expenses detected'. "
                "Return only the structured data, no additional text."
            )
            response = self.client.complete(
                messages=[
                    SystemMessage(content=prompt),
                    UserMessage(content=[
                        TextContentItem(text="Extract travel expenses from this receipt image."),
                        ImageContentItem(image_url=ImageUrl(url=image_url_str))
                    ])
                ],
                model=self.model_name,
                temperature=0.1,
                max_tokens=2048
            )
            extracted_text = response.choices[0].message.content
            return extracted_text
        except Exception as e:
            error_msg = f"[LOG] OCR Plugin: Error processing image: {str(e)}"
            print(error_msg)
            return error_msg

## खर्चों को संसाधित करना

खर्चों को संसाधित करने के लिए एक असिंक्रोनस फ़ंक्शन परिभाषित करें, जो आवश्यक एजेंटों को बनाकर और पंजीकृत करके उन्हें सक्रिय करता है।  
- यह फ़ंक्शन खर्चों को संसाधित करता है, पर्यावरण वेरिएबल्स को लोड करके, आवश्यक एजेंटों को बनाकर और उन्हें प्लगइन्स के रूप में पंजीकृत करके।  
- यह दो एजेंटों के साथ एक ग्रुप चैट बनाता है और खर्चों के डेटा के आधार पर ईमेल और पाई चार्ट बनाने के लिए एक प्रॉम्प्ट संदेश भेजता है।  
- यह चैट को सक्रिय करने के दौरान होने वाली किसी भी त्रुटि को संभालता है और एजेंटों की उचित सफाई सुनिश्चित करता है।  


In [6]:
async def process_expenses():
    load_dotenv()
    settings_slm = OpenAIChatPromptExecutionSettings(service_id="gpt-4o")
    settings_llm = OpenAIChatPromptExecutionSettings(service_id="open_ai")  # Fixed typo in service_id
    
    ocr_agent = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion("ocrAgent"),
        name="ocr_agent",
        instructions="Extract travel expense data from the receipt image in the prompt using the 'extract_text' function from the 'ocrAgent' plugin. Return the data in the format 'date|description|amount|category' separated by semicolons.",
        arguments=KernelArguments(settings=settings_slm)
    )
    
       
    email_agent = ChatCompletionAgent(
            kernel=_create_kernel_with_chat_completion("expenseEmailAgent"),
            name="email_agent",
            instructions="Take the travel expense data from the previous agent and generate a professional expense claim email using the 'generate_expense_email' function from the 'expenseEmailAgent' plugin, then pass the data forward.",
            arguments=KernelArguments(
                settings=settings_llm)
        )


    kernel = Kernel()

    # Use fixed path to receipt.jpg in the same folder
    image_path = "./receipt.jpg"
    
    # Create a structured message with text and image content for OCR processing
    image_url_str = f"file://{image_path}"
    
    # Using the correct format for multi-modal content
    user_message = ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text="""
            Please extract the raw text from this receipt image, focusing on travel expenses like dates, descriptions, amounts, and categories (e.g., Transportation, Accommodation, Meals, Miscellaneous).
            Then generate a professional expense claim email.
                        """),
            ImageContent.from_image_file(path=image_path)
        ]
    )

    # Register plugins with the kernel
    kernel.add_plugin(OCRAgentPlugin(), plugin_name="ocrAgent")
    kernel.add_plugin(ExpenseEmailAgent(), plugin_name="expenseEmailAgent")

    # Create group chat
    chat = AgentGroupChat(
        agents=[ocr_agent, email_agent],
        selection_strategy=SequentialSelectionStrategy(initial_agent=ocr_agent),
        termination_strategy=DefaultTerminationStrategy(maximum_iterations=1)
    )

    # Add user message with prompt
    await chat.add_chat_message(user_message)
    print(f"# User message added to chat with receipt image")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")


## मुख्य फ़ंक्शन

मुख्य फ़ंक्शन को परिभाषित करें ताकि कंसोल साफ़ किया जा सके और `process_expenses` फ़ंक्शन को असिंक्रोनस रूप से चलाया जा सके।


In [9]:
async def main():
    # Clear the console
    os.system('cls' if os.name=='nt' else 'clear')

    # Run the async agent code
    await process_expenses()

await main()

# User message added to chat with receipt image
# Agent - ocr_agent: 'The receipt primarily seems to capture costs for meals and beverages. Below is the extracted travel expense data:

**Travel Expense Data:**  
`2 May '22|Meals at restaurant|75.15|Meals`

---

**Professional Expense Claim Email Draft:**  

**Subject:** Expense Claim for Meals – 2 May 2022  

Dear [Recipient's Name],  

I am submitting an expense claim for a meal incurred during a business-related trip. Below are the details:  

- **Date:** 2 May 2022  
- **Expense Description:** Meals at a restaurant  
- **Amount:** $75.15  
- **Category:** Meals  

Please find the attached receipt for your reference. Kindly process the reimbursement at your earliest convenience. Let me know if you require additional information.  

Thank you for your assistance.  

Best regards,  
[Your Name]  
[Your Contact Information]  

Let me know if you need further revisions or additional details!'



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल दस्तावेज़, जो इसकी मूल भाषा में है, को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
